# Decision Trees  

In [11]:
import numpy as np
import pandas as pd

`eps` is the smallest representable number. At times, we get log(0) or 0 in the denominator, to avoid that we are going to use this.  

In [12]:
eps = np.finfo(float).eps
eps

2.220446049250313e-16

In [13]:
Age = 'junior,middle,senior,senior,middle,junior,junior,middle,middle,junior,junior,senior,middle,junior,junior,senior,senior'.split(',')
Married = 'yes,no,no,no,yes,no,yes,yes,no,no,no,yes,yes,no,no,yes,yes'.split(',')
Salary = 'high,low,low,low,high,high,low,high,low,low,high,low,high,high,high,high,high'.split(',')
Home_owner = 'yes,yes,no,yes,yes,yes,yes,no,no,no,yes,no,yes,yes,no,yes,no'.split(',')
Loan_worthy = 'yes,no,no,no,yes,yes,yes,yes,no,no,no,yes,yes,yes,no,yes,yes'.split(',')

In [14]:
table = {'Age':Age, 'Married':Married, 'Salary':Salary,'Home_owner':Home_owner,'Loan_worthy':Loan_worthy}
Loan_data = pd.DataFrame(data=table,columns = table)

`play_tennis is the data used in this lecture video (see below)`   

In [15]:
Outlook = ['Sunny','Sunny','Overcast','Rainy','Rainy','Rainy','Overcast','Sunny','Sunny','Rainy','Sunny','Overcast','Overcast','Rainy']
Temperature = ['Hot','Hot','Hot','Mild','Cool','Cool','Cool','Mild','Cool','Mild','Mild','Mild','Hot','Mild']
Humidity = ['High','High','High','High','Normal','Normal','Normal','High','Normal','Normal','Normal','High','Normal','High']
Wind = ['False','True','False','False','False','True','True','False','False','False','True','True','False','True']
Play = ['No','No','Yes','Yes','Yes','No','Yes','No','Yes','Yes','Yes','Yes','Yes','No']
table1 = {'Outlook':Outlook, 'Temperature':Temperature, 'Humidity':Humidity,'Wind':Wind,'Play':Play}
play_tennis = pd.DataFrame(data=table1,columns=table1)

In [16]:
play_tennis

,Outlook,Temperature,Humidity,Wind,Play
0,Sunny,Hot,High,False,No
1,Sunny,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Rainy,Mild,High,False,Yes
4,Rainy,Cool,Normal,False,Yes
5,Rainy,Cool,Normal,True,No
6,Overcast,Cool,Normal,True,Yes
7,Sunny,Mild,High,False,No
8,Sunny,Cool,Normal,False,Yes
9,Rainy,Mild,Normal,False,Yes


In [17]:
play_tennis.shape

(14, 5)

In [18]:
play_tennis.values

array([['Sunny', 'Hot', 'High', 'False', 'No'],
       ['Sunny', 'Hot', 'High', 'True', 'No'],
       ['Overcast', 'Hot', 'High', 'False', 'Yes'],
       ['Rainy', 'Mild', 'High', 'False', 'Yes'],
       ['Rainy', 'Cool', 'Normal', 'False', 'Yes'],
       ['Rainy', 'Cool', 'Normal', 'True', 'No'],
       ['Overcast', 'Cool', 'Normal', 'True', 'Yes'],
       ['Sunny', 'Mild', 'High', 'False', 'No'],
       ['Sunny', 'Cool', 'Normal', 'False', 'Yes'],
       ['Rainy', 'Mild', 'Normal', 'False', 'Yes'],
       ['Sunny', 'Mild', 'Normal', 'True', 'Yes'],
       ['Overcast', 'Mild', 'High', 'True', 'Yes'],
       ['Overcast', 'Hot', 'Normal', 'False', 'Yes'],
       ['Rainy', 'Mild', 'High', 'True', 'No']], dtype=object)

In [19]:
play_tennis.keys()

Index(['Outlook', 'Temperature', 'Humidity', 'Wind', 'Play'], dtype='object')

In [20]:
target = play_tennis.keys()[-1]     # Name of the target column
target

'Play'

In [21]:
play_tennis.keys()[:-1]     # All columns excluding the target

Index(['Outlook', 'Temperature', 'Humidity', 'Wind'], dtype='object')

Let's check total number of labels.  

In [22]:
play_tennis[target].unique()

array(['No', 'Yes'], dtype=object)

There are two labels `Yes` and `No`  

In [24]:
print(play_tennis[target].value_counts()[play_tennis[target].unique()[0]])
print(play_tennis[target].value_counts()[play_tennis[target].unique()[1]])

5
9


5/14 examples have label `No` and the remaining 9/14 examples have label `Yes`  

## Calculating the entropy of the whole dataset  

Since the formula for information gain requires entropy of the whole dataset, we compute that now:  

In [25]:
def find_entropy_whole(df):

    # last column in dataframe is target variable.
    target = df.keys()[-1]

    # initialization
    overall_entropy = 0

    # possible values of target  
    values_in_target = df[target].unique()

    for value in values_in_target:
        p = df[target].value_counts()[value] / len(df[target])
        overall_entropy += -p * np.log2(p)
    return overall_entropy

find_entropy_whole(play_tennis)

0.9402859586706311

## Calculating entropy of an attribute  

In [26]:
def find_entropy_of_attribute(df, attribute):

    # last column in dataframe is the target variable
    target = df.keys()[-1]

    # possible values of target
    values_in_target = df[target].unique()

    # this gives different features in that attribute (like 'hot', 'cold' in temperature)
    values_in_attribute = df[attribute].unique()

    # initialize attribute's entropy
    entropy_attribute = 0

    for value_in_attribute in values_in_attribute:
        overall_entropy = 0

        for value_in_target in values_in_target:
            num = len(df[attribute][df[attribute] == value_in_attribute][df[target] == value_in_target])
            den = len(df[attribute][df[attribute] == value_in_attribute])
            p = num/(den+eps)
            overall_entropy += -p * np.log2(p+eps)
        p2 = den/len(df)
        entropy_attribute += -p2*overall_entropy
    return abs(entropy_attribute)

Let's compute entropy of different attributes now:  

In [29]:
for i_attribute in play_tennis.keys()[:-1]:
    print(f"Entropy of attribute \'{i_attribute}\' is: {find_entropy_of_attribute(play_tennis, i_attribute)}")

Entropy of attribute 'Outlook' is: 0.6935361388961914
Entropy of attribute 'Temperature' is: 0.9110633930116756
Entropy of attribute 'Humidity' is: 0.7884504573082889
Entropy of attribute 'Wind' is: 0.892158928262361


## Finding the best attribute  

In [30]:
def find_best_attribute_to_divide(df):

    # information gain
    IG = []
    
    # all column names
    all_attribute_names = df.keys()[:-1]

    for attribute in all_attribute_names:
        # compute information gain for every attribute
        IG.append(find_entropy_whole(df) - find_entropy_of_attribute(df, attribute))

    # get the index of attribute with best information gain
    index_of_attribute_with_max_IG = np.argmax(IG)

    best_attribute = all_attribute_names[index_of_attribute_with_max_IG]

    return best_attribute

find_best_attribute_to_divide(play_tennis)

'Outlook'

## Building the Decision Tree  

In [35]:
def buildTree(df, tree=None):
    # last column is dataframe
    target = df.keys()[-1]

    # Here we build our decision tree

    # Get attribute with maximum information gain
    node = find_best_attribute_to_divide(df)

    # Get distinct value of that attribute
    attValue = np.unique(df[node])

    # Create an empty dictionary to create tree
    if tree is None:
        tree = {}
        tree[node] = {}

    # We make a loop to construct a tree by calling this function recursively.
    # In this, we check if the subset is pure and stop if it is pure.
    for value in attValue:
        subtable = df[df[node] == value].reset_index(drop=True)
        c1Value, counts = np.unique(subtable['Play'], return_counts=True)

        if len(counts) == 1:        # Checking purity of subset
            tree[node][value] = c1Value[0]
        else:
            tree[node][value] = buildTree(subtable)    # Calling the function recursively.

    return tree

buildTree(play_tennis)

{'Outlook': {'Overcast': 'Yes',
  'Rainy': {'Wind': {'False': 'Yes', 'True': 'No'}},
  'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}

*ID3 in its pure form performs no backtracking in its search. Once it selects an attribute to test at a particular level in the tree, it never backtracks to consider this choice. Therefore, it is susceptible to the usual risks of hill-climbing search without backtracking: converging to locally optimal solutions that are not globally optimal.* 